In [ ]:
Transformer + CNN + Bi-LSTM + Attention

In [ ]:
# 🎯 Install dependencies
!pip install -q transformers tensorflow xgboost scikit-learn

# 📚 Imports
import numpy as np, pandas as pd, gc
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv1D, GlobalMaxPooling1D,
                                     Bidirectional, LSTM, Dense, Dropout,
                                     Attention, Layer, Concatenate)
from tensorflow.keras.models import Model
from transformers import TFAutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import xgboost as xgb

# 🧭 1. Load data
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/thesis/thesis 2.csv')
df = df[['Text','Label']].dropna()
X, y = df['Text'].tolist(), df['Label'].values
Xt, Xv, yt, yv = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 🧠 2. Transformer embeddings
MODEL = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
bert = TFAutoModel.from_pretrained(MODEL)

def get_emb(texts, max_len=128, bs=16):
    out = []
    for i in range(0, len(texts), bs):
        b = texts[i:i+bs]
        tk = tokenizer(b, padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')
        h = bert(tk['input_ids'], attention_mask=tk['attention_mask']).last_hidden_state
        out.append(h.numpy()); del tk, h; gc.collect()
    return np.vstack(out)

Xt_emb, Xv_emb = get_emb(Xt), get_emb(Xv)

# 🎯 3. Attention layer
class SelfAtt(Layer):
    def __init__(self, units): super().__init__(); self.Wq=Dense(units); self.Wk=Dense(units); self.Wv=Dense(units); self.att=Attention()
    def call(self, x):
        return self.att([self.Wq(x), self.Wk(x), self.Wv(x)])

# 🧱 4. Dual-branch model
def get_dual_out(inp):
    # CNN branch
    x = Conv1D(128, 3, activation='relu', padding='same')(inp)
    x = GlobalMaxPooling1D()(x)
    x = Dense(64, activation='relu')(x)
    cnn = Dropout(0.3)(x)
    # BiLSTM + Attention branch
    y = Bidirectional(LSTM(64, return_sequences=True))(inp)
    y = Dropout(0.3)(y)
    a = SelfAtt(64)(y)
    a = GlobalMaxPooling1D()(a)
    lstm = Dense(64, activation='relu')(a)
    return cnn, lstm

inp = Input(shape=(128, bert.config.hidden_size))
c_out, l_out = get_dual_out(inp)
merged = Concatenate()([c_out, l_out])
feat_model = Model(inp, merged)

# 🔍 5. Extract features
Xtf = feat_model.predict(Xt_emb, batch_size=16, verbose=1)
Xvf = feat_model.predict(Xv_emb, batch_size=16, verbose=1)

# 🛠 6. Train XGBoost and evaluate
clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
clf.fit(Xtf, yt)
yv_pred = clf.predict(Xvf)

print("Accuracy:", accuracy_score(yv, yv_pred))
print("Precision:", precision_score(yv, yv_pred, average='weighted'))
print("Recall:", recall_score(yv, yv_pred, average='weighted'))
print("F1 Score:", f1_score(yv, yv_pred, average='weighted'))
print("Confusion Matrix:\n", confusion_matrix(yv, yv_pred))
